#📘 Задание: Обучение моделей сегментации на реальных данных

В этом задании вы реализуете полный цикл обучения модели **бинарной семантической сегментации**: от подготовки датасета до обучения, сохранения лучшей модели и демонстрации инференса.


## Датасет: HoofedAnimals (binary)
Архив с данными размещён в учебном репозитории:

- `HoofedAnimals.zip`: https://raw.githubusercontent.com/Mike030668/cv_course_muiv/main/data/HoofedAnimals.zip

Изображения: `.pgm` (градации серого), маски: `.ppm`.
Для **binary** сегментации используйте правило: **любой ненулевой пиксель маски = 1 (foreground)**, иначе 0 (background).


## Что нужно получить на выходе
1. Графики обучения (**train/val**): `loss`, `IoU`, `Dice` (и при желании `pixel accuracy`).
2. Сохранение **лучшего чекпоинта** (по `val Dice` или `val IoU`).
3. Демонстрация инференса на валидации: `image / target / prob / prediction`.
4. Сравнение двух подходов:
   - **своя модель (from scratch)**
   - **fine-tune предобученной segmentation-модели из библиотеки**


---

# 🟦 Этап 1. Загрузка и исследование датасета
**Задачи:**
- скачать `HoofedAnimals.zip` из репозитория (как в примере классификации);
- распаковать в отдельную папку;
- найти пары `image–mask` и вывести несколько примеров с наложением маски;
- проверить, что бинаризация даёт значения маски `{0,1}`.

**Подсказка:** ориентируйтесь на логику из `Train_segmentation.ipynb` и решения `task_done.ipynb`:
- “robust pairing” (сопоставление файлов по stem),
- `show_samples()` для визуального контроля.


---

# 🟩 Этап 2. Dataset / DataLoader для сегментации
**Задачи:**
- реализовать класс `Dataset`, который возвращает `(image_tensor, mask_tensor)`;
- реализовать согласованные transforms для изображения и маски:
  - `resize` (например, 256×256),
  - `horizontal flip` для train,
  - `to_tensor` для изображения,
  - бинаризация маски;
- собрать `DataLoader` для train/val.

**Важно:** для маски используйте `NEAREST` интерполяцию при resize, иначе появятся промежуточные значения.


---

# 🟨 Этап 3. Своя модель сегментации (from scratch)
**Задачи:**
- реализовать компактную архитектуру (например, mini U-Net или encoder–decoder);
- выбрать loss: `BCEWithLogitsLoss`;
- реализовать вычисление метрик `IoU`, `Dice`, `pixel accuracy`.

**Подсказка:** в качестве основы используйте код и структуру из `Train_segmentation.ipynb`:
- `UNetSmall` / `UNet`-подобная модель,
- `metrics_from_logits(logits, target)`.


---

# 🟧 Этап 4. “Максимальное” обучение (графики, scheduler, best checkpoint)
**Задачи:**
- реализовать цикл обучения с функциями:
  - `train_one_epoch(...)`
  - `evaluate(...)`
  - `plot_history(...)`
  - `save_checkpoint(...)` / `load_checkpoint(...)`
- добавить scheduler `ReduceLROnPlateau` (step после валидации);
- сохранять лучший чекпоинт по `val Dice` (или `val IoU`);
- построить графики train/val.

**Подсказка:** просто адаптируйте готовые функции из `Train_segmentation.ipynb` (они уже подходят под binary).


---

# 🟥 Этап 5. Fine-tune предобученной segmentation модели
**Задачи:**
- взять модель из `torchvision.models.segmentation` (например, `deeplabv3_resnet50`);
- заменить последний классификатор под **1 канал** (binary logits);
- использовать preprocessing, соответствующий весам (обычно ImageNet mean/std); 
- обучить модель тем же “максимальным” пайплайном (графики, scheduler, best checkpoint);
- визуально сравнить инференс своей модели и fine-tuned модели.

**Подсказка:** смотрите реализацию `DeeplabBinary` и `SegTransformNorm` в `Train_segmentation.ipynb` и `task_done.ipynb`.
Для корректного отображения нормализованных изображений используйте **denorm** перед `imshow`.


---

# 🟪 Этап 6. Финальный анализ
1. Сравните модели по:
   - финальным значениям `val IoU` / `val Dice`,
   - скорости обучения (как быстро выходят на плато),
   - качеству предсказаний на примерах (типичные ошибки).
2. Сделайте краткие выводы:
   - что даёт fine-tune относительно обучения “с нуля”; 
   - где модель ошибается и почему (малые объекты, размытые границы, фоновые текстуры).
